In [ ]:
import cv2
import numpy as np

# Function for noise reduction
def denoise(image):
    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    return blurred_image

# Function for contrast enhancement
def enhance_contrast(image):
    # Convert image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # Split the LAB image into channels
    l, a, b = cv2.split(lab)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    
    # Merge the CLAHE enhanced L channel with the original A and B channels
    enhanced_lab = cv2.merge((cl, a, b))
    
    # Convert LAB image back to BGR color space
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    
    return enhanced_image

# Load the full image
full_image = cv2.imread('photo_16.jpg')

new_width = 345.6
new_height = 460.8

# Resize the image
resized_image = cv2.resize(full_image, (int(new_width), int(new_height)))

# Check if the image was loaded successfully
if resized_image is None:
    print("Error: Unable to load the image. Please check the file path and format.")
else:
    # Preprocess the image
    preprocessed_image = denoise(resized_image)
    preprocessed_image = enhance_contrast(preprocessed_image)
    
    # Calculate the height for the cropped region (half of original height)
    crop_height = preprocessed_image.shape[0] // 2
    
    # Define the coordinates for the cropped region (from top to bottom)
    x1 = 0
    x2 = preprocessed_image.shape[1]
    y1 = 10
    y2 = crop_height
   
    # Perform the crop
    cropped_image = preprocessed_image[y1:y2, x1:x2]
  

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # Apply a threshold to segment the fungus part
    _, binary_image = cv2.threshold(gray_image,50, 255, cv2.THRESH_BINARY)

    # Invert the binary image
    binary_image = cv2.bitwise_not(binary_image)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to separate the fungus part
    mask = np.zeros_like(cropped_image)

    # Draw contours on the mask
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Apply the mask to the original image
    segmented_image = cv2.bitwise_and(cropped_image, mask)
    cv2.imwrite('segmented_image with filter_16.jpg',segmented_image)
    cv2.imwrite('Cropped_Image with filters_16.jpg', cropped_image)
    # Display the segmented image
    cv2.imshow('Segmented Image', segmented_image)

    
    cv2.imshow('Cropped_Image with filters_', cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
